In [ ]:
from CircleNet.system import Simulation
from CircleNet.shape import circle,Timer,get_log_normal
from CircleNet.shanghai_platform import MatchingPlatform, Driver, Passenger
from CircleNet.shanghai_platform_extension import extract_result_data,vks_detour
import numpy as np
import time


#TODO
#put writting functions somewhere else
#put extraction functions somewhere else
#reorganise parameters etc in other file
#controlleur de temps

In [ ]:
#automatic functions


class Write_csv():
    def __init__(self,name,keys_to_save):
        self.file_name=name
        self.the_string="{"+"},{".join(keys_to_save)+"}\n"
        with open(self.file_name , "w") as file :
            file.write(",".join(keys_to_save)+"\n")#write the first line
            assert len(keys_to_save)>0, "nothing to save"
    def write(self,**values):
        with open(self.file_name , "a") as file :
            file.write(self.the_string.format(**values))
 


In [ ]:


def save_agents(simu,*files_info):
    """every file in files_info is a dictionnary
        file[name] -> name of the output file (preceded by {id}_)
        file[selection_function] -> send true for all agents concerned by this file
        file[extract_info] -> function that extract information from an agent (put in a dictionnary)
        file[info_list] -> list of info to store"""
    for file in files_info:
        file["writter"]=open("data/{}_{}.csv".format(simu.id_sim,file["name"] ),"w")
        file["writter"].write(",".join(file["info_list"])+"\n")
        file["the_string"]="{"+"},{".join(file["info_list"])+"}\n"
    for agent in simu:
        for file in files_info:
            if file["selection_function"](agent):
                data=file["extract_info"](agent,simu)
                file["writter"].write(file["the_string"].format(**data))
    for file in files_info:
        file["writter"].close()
    

In [ ]:
def extract(module):
    static={}
    variable={}
    for key in module.__dir__():
        if "__" not in key:
            if key[0] is "V":
                variable[key[1:]]=module.__dict__[key]
            else:
                static[key]=module.__dict__[key]
    return static,variable


def get_parameter_sets(static,**variable):
    if not variable:#everything is static
        yield static
        raise StopIteration
    key,new_variable = variable.popitem()# new_variable is the list of values that key should take
    for value in new_variable:
        static[key]=value
        for out in get_parameter_sets(static,**variable):
            yield out
            
def get_parameters(module_name):
    s,v=extract(module_name)
    return get_parameter_sets(s,**v)

In [ ]:
#SIMULATION CREATOR
def create_simulation(speed,radius,end,first_watching_before_first_departure,window_size_of_departure,
                       time_elasticity,fuel_cost,watching_repetition_average,watching_repetition_variance,
                      time_perception_average,time_perception_variance,publishing_advance,benefits,N_driver,N_passenger):
    
    
    #random functions
    watching_repetition=get_log_normal(watching_repetition_average,watching_repetition_variance)
    time_perception=get_log_normal(time_perception_average,time_perception_variance)
    

    #agents generator
    def SimpleDriver(simulation):
        t=simulation.timer.random_time()
        O=simulation.network.position_generator()
        D=simulation.network.position_generator()
        w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
        A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
        return Driver(first_watching_time=t,
                      repetition_time=watching_repetition(),
                      departure_window=w,
                      position=O,destination=D,
                      last_arrival_time=A,
                      fuel_cost=fuel_cost,
                      time_perception=time_perception())
    def SimplePassenger(simulation):
        t=simulation.timer.random_time()
        O=simulation.network.position_generator()
        D=simulation.network.position_generator()
        return Passenger(publishing_time=t,
                         last_departure_time=t+publishing_advance,
                         position=O,destination=D)
    
    N=circle(radius,speed)
    T=Timer(end)
    simu=Simulation(N,T)
    simu.matchingAlgo=MatchingPlatform(benefits,simu)
    for i in range(N_driver):
        simu.add(SimpleDriver(simu))
    for i in range(N_passenger):
        simu.add(SimplePassenger(simu))
    return simu

In [ ]:
#definition of savers :

#general
to_save=['id_sim', 'nb_passenger','nb_driver','execution_time', 'nb_match','driver_efficiency', 'passenger_efficiency',  
         'average_waiting_time','average_vks', 'total_vks']
result_file=Write_csv("data/general2.csv",to_save)

#passengers
def extract_passenger_info(passenger,simu):
    m=0
    d,w="",""
    for t,action in passenger.story:
        if action[0] is "matched":
            m=1
            d=action[1]["driver"]
            w=t
            break
    if m is 1:#has to compute a waiting time
        for t,action in passenger.story:
            if action[0] is "waiting":
                w-=t
    out={"id_num":passenger.id_number,"departure_t":passenger.last_departure_time,"matched":m,"driver":d,"waiting_time":w}
    out["Ox"],out["Oy"]=passenger.position
    out["Dx"],out["Dy"]=passenger.destination
    return out
file_passenger={"name":"passengers","selection_function":lambda x:isinstance(x,Passenger),
                "extract_info":extract_passenger_info,"info_list":["id_num","Ox","Oy","Dx","Dy","departure_t","matched","driver","waiting_time"]}

#drivers
def extract_driver_info(driver,simu):
    m=0
    p,v,d="","",""
    for _,action in driver.story:
        if action[0] is "matched":
            m=1
            p=action[1]["passenger"]
            break
    if m is 1:#do a detour
        route=[]
        for _,action in driver.story.reverse_iter():
            if action[0] is "moving" :
                route.append(action[1]["start"])
            elif action[0] is "arrived":
                route.append(action[1]["point"])
        v,d=vks_detour(route,simu.network)
    out={"id_num":driver.id_number,"departure_t":driver.departure_window[1],"matched":m,"passenger":p,
         "value_of_time":driver.time_perception,"watching_repetition_time":driver.repetition_time,"trip_vks":v,"detour":d}
    out["Ox"],out["Oy"]=driver.position
    out["Dx"],out["Dy"]=driver.destination
    return out
file_driver={"name":"drivers","selection_function":lambda x:isinstance(x,Driver),
                "extract_info":extract_driver_info,
                "info_list":["id_num","Ox","Oy","Dx","Dy","departure_t","value_of_time","watching_repetition_time","matched","passenger","trip_vks","detour"]}



In [ ]:
#execution

import config as parameter_file

for PARAMETERS in get_parameters(parameter_file):
    print("begining of simulation with ",PARAMETERS["N_driver"]," drivers and ",PARAMETERS["N_passenger"]," passengers")
    #construction
    simu=create_simulation(**PARAMETERS)
    #execution
    simu()
    results=extract_result_data(simu)
    #saving
    result_file.write(**results)
    save_agents(simu,file_passenger,file_driver)
    
